Boilerplate

In [29]:
import $ivy.`edu.berkeley.cs::chisel3:3.0-SNAPSHOT_2017-07-19` 
import $ivy.`edu.berkeley.cs::chisel-iotesters:1.1-SNAPSHOT_2017-07-19`
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

class OperatorsExercise extends Module {
  val io = IO(new Bundle {
    val seed = Input(UInt(4.W))
    val result = Output(Bool())
  })

  // YOUR CODE GOES HERE
  // Remember: seed and result are inside io, so access them with io.seed and io.result
  // Also, there's no need to typecast between 1-bit UInts and Bools 
  io.result := io.seed(3) ^ io.seed(2)
}

import $ivy.$                                                  

import $ivy.$                                                          

import chisel3._

import chisel3.util._

import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}


defined class OperatorsExercise

# Sequential Logic

_In the previous section, we built the combinational part of the LFSR - that is, given the simultaneous output of all the shift register bits, generate the bit to feed back in. In this section, we will build the sequential part, the shift registers themselves._

The basic stateful element in Chisel is the register, or `Reg`. It keeps its current value through a clock cycle, and can optionally be updated with a new value that becomes visible at the next clock edge.

A register can be constructed given a data type. For instance, a 2-bit wide register without initialization:
```scala
val myReg = Reg(UInt(2.W))
```

or with initialization (to 1 decimal):
```scala
val myReg = RegInit(UInt(2.W), 1.U)
```

Its value can be updated with the `:=` operator. For example, this increments the register's value:
```scala
myReg := myReg + 1.U
```

Note that when read, it gives its current (stored) value, but the assignment doesn't take effect until the next clock edge.

Below is an example which uses a register to implement a 2-bit wide counter, initializing at 1 and overflowing from 3 to zero. Run the block and take a look at the output trace.

In [22]:
// Module containing the register
class MyCounter extends Module {
  val io = IO(new Bundle {
    val out = Output(UInt(2.W))
  })
  val myReg = RegInit(UInt(2.W), 1.U)
  myReg := myReg + 1.U
  io.out := myReg
    
  printf("out=%d\n", io.out)
}

// Testvector
class MyCounterTester(c: MyCounter) extends PeekPokeTester(c) {
  expect(c.io.out, 1)
  step(1)
  expect(c.io.out, 2)
  step(1)
  expect(c.io.out, 3)
  step(1)
  expect(c.io.out, 0)
  step(1)
  expect(c.io.out, 1)
  step(1)
}

// Driver run invocation
Driver(() => new MyCounter, "firrtl") {
  c => new MyCounterTester(c)
}

[info] [0.000] Elaborating design...
[info] [0.003] Done elaborating.
Total FIRRTL Compile Time: 8.1 ms
Total FIRRTL Compile Time: 10.7 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1501614765147
out=2
out=3
out=0
out=1
out=2
test cmd21WrapperHelperMyCounter Success: 5 tests passed in 10 cycles taking 0.003846 seconds
[info] [0.003] RAN 5 CYCLES PASSED


defined class MyCounter
defined class MyCounterTester
res21_2: Boolean = true

## Now you try
Given that, now build a module that implements a shift register for your LFSR. Specifically:
- Each element is a single bit wide.
- Has 4 stages.
- Takes a single input bit, which is the next value into the shift register.
- Outputs the parallel output of the shift register, with the most significant bit being the last element of the shift register and the least significant bit being the first element of the shift register. `Cat` may come in handy.
- The output initializes at `b0001`.
- Shifts each clock cycle (no enable signal).

A basic Module skeleton, testvector, and Driver invocation is provided below. The first register has been provided for you.

In [47]:
class MyShiftRegister extends Module {
  val io = IO(new Bundle {
    val in = Input(Bool())
    val out = Output(UInt(4.W))
  })
  val s1 = RegInit(Bool(), true.B)
    
  // YOUR CODE HERE
  // don't forget to update the first register!
    
  s1 := io.in
  val s2 = RegInit(Bool(), false.B)
  s2 := s1
  val s3 = RegInit(Bool(), false.B)
  s3 := s2
  val s4 = RegInit(Bool(), false.B)
  s4 := s3
    
  io.out := Cat(s4, s3, s2, s1)
  printf("%x\n", io.out)
}

defined class MyShiftRegister

In [48]:
class MyShiftRegisterTester(c: MyShiftRegister) extends PeekPokeTester(c) {
  expect(c.io.out, 1)  // b0001
  poke(c.io.in, 0)
  step(1)
  expect(c.io.out, 2)  // b0010
  poke(c.io.in, 1)
  step(1)
  expect(c.io.out, 5)  // b0101
  poke(c.io.in, 1)
  step(1)
  expect(c.io.out, 11)  // b1011
  poke(c.io.in, 0)
  step(1)
  expect(c.io.out, 6)  // b0110
}

Driver(() => new MyShiftRegister, "firrtl") {
  c => new MyShiftRegisterTester(c)
}

[info] [0.000] Elaborating design...
[info] [0.002] Done elaborating.
Total FIRRTL Compile Time: 6.7 ms
Total FIRRTL Compile Time: 6.8 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1501698597918
2
5
b
6
test cmd46WrapperHelperMyShiftRegister Success: 5 tests passed in 9 cycles taking 0.005147 seconds
[info] [0.005] RAN 4 CYCLES PASSED


defined class MyShiftRegisterTester
res47_1: Boolean = true

Chisel also offers several more variants of the register constructor, for example with a `next` field or with an `enable` signal, but these are outside the scope of this introductory tutorial.

# Conditional Logic and Wires
_We've now seen the basics of building combinational and sequential logic circuits, but those are very low-level primitives. Chisel provides a higher level of abstraction for some operations._

## Conditional Register Updates
Chisel operators can be gated by putting them inside a `when` conditional block. In our counter example above, if we also added a `count` input signal, we could gate the incrementing of the counter as follows:
```scala
when (io.count) {
  myReg := myReg + 1.U
}
```

## Wires
Conditional blocks can also be used to generate combinational logic. While we've seen the update operator `:=` used combinationally on Module boundary `IO`s, it can be also applied to Module-internal `Wire`s. Here's an example of the counter above, styled slightly differently to make the combinational logic explicit and using `Wire`s.
```scala
val myReg = RegInit(UInt(2.W), 1.U)
val nextReg = Wire(UInt(2.W))  // constructed similarly to Regs
nextReg := myReg + 1.U
myReg := nextReg
```

## Conditional Combinational Updates
Let's say now that we want a counter that counts 0, 1, 2, then resets. In this case, since we need to reset on 2, we can no longer depend on the implicit integer overflow and need to make the logic explicit:
```scala
val nextReg = Wire(UInt(2.W))
when (myReg === 2.U) {
  nextReg := 0.U
} .otherwise {
  nextReg := myReg + 1.U
}
```

As you can see from the example above, `when`-`otherwise` blocks behave like `if`-`else` blocks in Scala, but generate conditional hardware. `when` blocks also have an `.elsewhen (condition)`, much like `else if (condition)`.

_Note that there is a `.` before the `elsewhen`, this due to limitations of Scala syntax._

When there are multiple update operators that can apply at the same time, the last one takes priority. So the above example can be equivalently rewritten by unconditionally updating `nextReg` with the counter increment, then overriding it in the overflow case:

```scala
val nextReg = Wire(UInt(2.W))
nextReg := myReg + 1.U
when (myReg === 2.U) {
  nextReg := 0.U
}
```

There is also `WireInit`, a shorthand which declares and initializes a `Wire`:

```scala
val nextReg = WireInit(UInt(2.W), myReg + 1.U)
when (myReg === 2.U) {
  nextReg := 0.U
}
```

Whether to use the first, second, or third forms is mainly a matter of style and readability. They are all functionally equivalent.

## A Gated, 3-tick Counter
Putting it all together, here's the full example:

In [24]:
// Module containing the register
class MyNewCounter extends Module {
  val io = IO(new Bundle {
    val count = Input(Bool())
    val out = Output(UInt(2.W))
  })
  val myReg = RegInit(UInt(2.W), 1.U)

  val nextReg = Wire(UInt(2.W))
  when (myReg === 2.U) {
    nextReg := 0.U
  } .otherwise {
    nextReg := myReg + 1.U
  }
  
  when (io.count) {
    myReg := nextReg
  }

  io.out := myReg
    
  printf("out=%d\n", io.out)
}

// Testvector
class MyNewCounterTester(c: MyNewCounter) extends PeekPokeTester(c) {
  poke(c.io.count, 1)
  expect(c.io.out, 1)
  step(1)
  expect(c.io.out, 2)
  step(1)
  expect(c.io.out, 0)
  step(1)
  expect(c.io.out, 1)
  poke(c.io.count, 0)  // test pause
  step(1)
  expect(c.io.out, 1)
  step(1)
  expect(c.io.out, 1)
  poke(c.io.count, 1)  // unpause
  step(1)
  expect(c.io.out, 2)
  step(1)
}

// Driver run invocation
Driver(() => new MyNewCounter, "firrtl") {
  c => new MyNewCounterTester(c)
}

[info] [0.000] Elaborating design...
[info] [0.004] Done elaborating.
Total FIRRTL Compile Time: 20.9 ms
Total FIRRTL Compile Time: 10.1 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1501637761162
out=2
out=0
out=1
out=1
out=1
out=2
out=0
test cmd23WrapperHelperMyNewCounter Success: 7 tests passed in 12 cycles taking 0.005007 seconds
[info] [0.004] RAN 7 CYCLES PASSED


defined class MyNewCounter
defined class MyNewCounterTester
res23_2: Boolean = true

## Now you try

Take your shift register design from above, but modify it so that it only shifts when the update signal is asserted.

As with before, a basic testbench and module skeleton is provided for you.

In [25]:
class MyGatedShiftRegister extends Module {
  val io = IO(new Bundle {
    val in = Input(Bool())
    val update = Input(Bool())
    val out = Output(UInt(4.W))
  })
  val s1 = RegInit(Bool(), true.B)
    
  // YOUR CODE HERE
  // don't forget to update the first register!
    
  val s2 = RegInit(Bool(), false.B)
  val s3 = RegInit(Bool(), false.B)
  val s4 = RegInit(Bool(), false.B)
  
  when (io.update) {
    s1 := io.in
    s2 := s1
    s3 := s2
    s4 := s3
  }
    
  io.out := Cat(s4, s3, s2, s1)
}

defined class MyGatedShiftRegister

In [26]:
class MyGatedShiftRegisterTester(c: MyGatedShiftRegister) extends PeekPokeTester(c) {
  poke(c.io.update, 1)
  expect(c.io.out, 1)  // b0001
  poke(c.io.in, 0)
  step(1)
  expect(c.io.out, 2)  // b0010
  poke(c.io.in, 1)
  step(1)
  expect(c.io.out, 5)  // b0101
  poke(c.io.update, 0)  // pause
  step(1)
  expect(c.io.out, 5)
  step(1)
  expect(c.io.out, 5)
  step(1)
  poke(c.io.update, 1)  // unpause
  poke(c.io.in, 1)
  step(1)
  expect(c.io.out, 11)  // b1011
  poke(c.io.in, 0)
  step(1)
  expect(c.io.out, 6)  // b0110
}

Driver(() => new MyGatedShiftRegister, "firrtl") {
  c => new MyGatedShiftRegisterTester(c)
}

[info] [0.000] Elaborating design...
[info] [0.004] Done elaborating.
Total FIRRTL Compile Time: 15.7 ms
Total FIRRTL Compile Time: 14.8 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1501638222188
test cmd24WrapperHelperMyGatedShiftRegister Success: 7 tests passed in 12 cycles taking 0.007392 seconds
[info] [0.006] RAN 7 CYCLES PASSED


defined class MyGatedShiftRegisterTester
res25_1: Boolean = true

# Modules
_Without re-use, any non-trivial RTL design wouldn't get very far. This goes into the basics of re-use with Chisel._

In all the example code, you've probably noticed that all your classes extend the `Module` class. Chisel `Module`s are very much like Verilog `module`s, they are instantiable blocks of logic that contain well-defined IO ports.

As all `Module`s are classes, they can be instantiated using the `new` object syntax. However, they also must be wrapped in a `Module(...)` call. For example, to instantiate the 4-state counter example way above:
```scala
val myModule = Module(new MyCounter())
```

You can then reference its IO objects in its parent class. For example, if you wanted to know when `myModule`'s count was 3, you could:
```scala
val isCount3 = myModule.io.out === 3.U
```

## A 4-cycle tick
This example below uses the `MyCounter` class from above to build a block that generates a one high cycle for every 4 clock cycles by comparing its output.

In [28]:
// Module containing the register
class MyTicker extends Module {
  val io = IO(new Bundle {
    val out = Output(Bool())
  })
  
  val myModule = Module(new MyCounter())
  io.out := myModule.io.out === 3.U
}

// Testvector
class MyTickerTester(c: MyTicker) extends PeekPokeTester(c) {
  expect(c.io.out, 0)  // counter at 1
  step(1)
  expect(c.io.out, 0)  // counter at 2
  step(1)
  expect(c.io.out, 1)  // counter at 3
  step(1)
  expect(c.io.out, 0)  // counter at 0
  step(1)
  expect(c.io.out, 0)  // counter at 1
  step(1)
  expect(c.io.out, 0)  // counter at 2
  step(1)
  expect(c.io.out, 1)  // counter at 3
  step(1)
}

// Driver run invocation
Driver(() => new MyTicker, "firrtl") {
  c => new MyTickerTester(c)
}

[info] [0.000] Elaborating design...
[info] [0.003] Done elaborating.
Total FIRRTL Compile Time: 14.0 ms
Total FIRRTL Compile Time: 9.8 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1501639504204
out=2
out=3
out=0
out=1
out=2
out=3
out=0
test cmd27WrapperHelperMyTicker Success: 7 tests passed in 12 cycles taking 0.004465 seconds
[info] [0.003] RAN 7 CYCLES PASSED


defined class MyTicker
defined class MyTickerTester
res27_2: Boolean = true

Modules are just one form of composition and re-use in Chisel, you'll learn more soon.

## Now You Try: The Full LFSR

Now, build a full LFSR by taking your `MyGatedShiftRegister` and `OperatorsExercise` modules and instantiating them in a parent module. As usual, the module skeleton and a testbench is provided for you.

In [51]:
class MyLfsr extends Module {
  val io = IO(new Bundle {
    val update = Input(Bool())
    val out = Output(UInt(4.W))
  })
  
  val myShifter = Module(new MyGatedShiftRegister())
  myShifter.io.update := io.update
  
  // YOUR CODE HERE
  // Don't forget to make the other needed connections to myShifter!
  
  val myOperations = Module(new OperatorsExercise())

  myOperations.io.seed := myShifter.io.out
  myShifter.io.in := myOperations.io.result
  io.out := myShifter.io.out
  
  printf("%x\n", io.out)
}


defined class MyLfsr

In [52]:
// Testvector
class MyLfsrTester(c: MyLfsr) extends PeekPokeTester(c) {
  poke(c.io.update, 1)
  expect(c.io.out, 1)
  step(1)
  expect(c.io.out, 2)
  step(1)
  expect(c.io.out, 4)
  step(1)
  expect(c.io.out, 9)
  step(1)
  expect(c.io.out, 3)
  step(1)
  expect(c.io.out, 6)
  step(1)
  expect(c.io.out, 13)
  step(1)
  expect(c.io.out, 10)
  step(1)
  expect(c.io.out, 5)
  step(1)
  expect(c.io.out, 11)
  poke(c.io.update, 0)  // pause
  step(1)
  expect(c.io.out, 11)
  step(1)
  expect(c.io.out, 11)
  poke(c.io.update, 1)  // unpause
  step(1)
  expect(c.io.out, 7)
  step(1)
  expect(c.io.out, 15)
  step(1)
  expect(c.io.out, 14)
  step(1)
  expect(c.io.out, 12)
  step(1)
  expect(c.io.out, 8)
  step(1)
  expect(c.io.out, 1)  // LFSR restarts
  step(1)
  expect(c.io.out, 2)
  step(1)
  expect(c.io.out, 4)
  step(1)
  expect(c.io.out, 9)
  step(1)
}

// Driver run invocation
Driver(() => new MyLfsr, "firrtl") {
  c => new MyLfsrTester(c)
}

[info] [0.000] Elaborating design...
[info] [0.004] Done elaborating.
Total FIRRTL Compile Time: 18.4 ms
Total FIRRTL Compile Time: 12.9 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1501698829312
2
4
9
3
6
d
a
5
b
b
b
7
f
e
c
8
1
2
4
9
3
test cmd50WrapperHelperMyLfsr Success: 21 tests passed in 26 cycles taking 0.010868 seconds
[info] [0.009] RAN 21 CYCLES PASSED


defined class MyLfsrTester
res51_1: Boolean = true